In [1]:
import pandas as pd
import numpy as np
import os
import sys
import polars as pl
import json
from joblib import Parallel, delayed
import deepchem
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import StratifiedGroupKFold, GroupKFold, KFold, GroupShuffleSplit, ShuffleSplit
import gc
import random
from functools import partial
import catboost
from catboost import FeaturesData, Pool, CatBoost, CatBoostClassifier, CatBoostRegressor, CatBoostRanker
import time
import math
from sklearn.metrics import average_precision_score

from functools import partial
import mapply

# Configure Polars 
cfg = pl.Config()
cfg.set_tbl_rows(20)
cfg.set_tbl_cols(50)

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/opt/conda/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some PyTorch models, missing a dependency. No module named 'tensorflow'


polars.config.Config

In [10]:
df = pl.scan_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/cv/v1/train.csv').select('index').collect().to_numpy()
df

array([[  363110],
       [  363118],
       [  363119],
       ...,
       [98414880],
       [98414881],
       [98414886]])

In [7]:
df = pl.scan_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/cv/v1/val.csv').collect()
df

index,label,subset
i64,i64,i64
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
5,0,1
6,0,1
7,0,1
8,0,1


In [8]:
arr = np.arange(0, 10)
np.array(arr)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [2]:
CHAR2IDX = {'l': 1, 'y': 2, '@': 3, '3': 4, 'H': 5, 'S': 6, 'F': 7, 'C': 8, 'r': 9,
            's': 10, '/': 11, 'c': 12, 'o': 13, '+': 14, 'I': 15, '5': 16, '(': 17,
            '2': 18, ')': 19, '9': 20, 'i': 21, '#': 22, '6': 23, '8': 24, '4': 25,
            '=': 26, '1': 27, 'O': 28, '[': 29, 'D': 30, 'B': 31, ']': 32, 'N': 33,
            '7': 34, 'n': 35, '-': 36}

In [3]:
max(CHAR2IDX.values()), min(CHAR2IDX.values()), len(CHAR2IDX)

(36, 1, 36)

In [4]:
train_df = pl.scan_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/train_v2.csv').select(
        pl.col('molecule'),
#         pl.col('bb1', 'bb2', 'bb3').cast(pl.UInt16),
        pl.col('BRD4', 'HSA', 'sEH').cast(pl.UInt8),
    ).collect()
train_df

molecule,BRD4,HSA,sEH
str,u8,u8,u8
"""C#CCOc1ccc(CNc…",0,0,0
"""C#CCOc1ccc(CNc…",0,0,0
"""C#CCOc1ccc(CNc…",0,0,0
"""C#CCOc1ccc(CNc…",0,0,0
"""C#CCOc1ccc(CNc…",0,0,0
"""C#CCOc1ccc(CNc…",0,0,0
"""C#CCOc1ccc(CNc…",0,0,0
"""C#CCOc1ccc(CNc…",0,0,0
"""C#CCOc1ccc(CNc…",0,0,0


In [5]:
test_df = pl.scan_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/test_v3.csv').select(
        pl.col('molecule'),
#         pl.col('bb1', 'bb2', 'bb3').cast(pl.UInt16),
        # pl.col('BRD4', 'HSA', 'sEH').cast(pl.UInt8),
    ).collect()
test_df

molecule
str
"""C#CCCC[C@H](Nc…"
"""C#CCCC[C@H](Nc…"
"""C#CCCC[C@H](Nc…"
"""C#CCCC[C@H](Nc…"
"""C#CCCC[C@H](Nc…"
"""C#CCCC[C@H](Nc…"
"""C#CCCC[C@H](Nc…"
"""C#CCCC[C@H](Nc…"
"""C#CCCC[C@H](Nc…"


In [6]:
tmp = [train_df['molecule'].str.len_chars().max(), test_df['molecule'].str.len_chars().max()]
print(tmp)
MAX_LEN = max(tmp)
MAX_LEN

[142, 127]


142

In [7]:
def get_char_idxs(smile):
    idxs = np.zeros((MAX_LEN,), dtype = np.uint8)
    for i, c in enumerate(smile):
        idxs[i] = CHAR2IDX[c]
    return idxs

get_char_idxs(train_df[0, 'molecule'])

array([ 8, 22,  8,  8, 28, 12, 27, 12, 12, 12, 17,  8, 33, 12, 18, 35, 12,
       17, 33,  8,  8,  4,  8,  8,  8, 33,  4, 12,  4, 12, 12, 12, 35, 35,
        4, 19, 35, 12, 17, 33, 29,  8,  3,  3,  5, 32, 17,  8,  8, 22,  8,
       19,  8,  8, 17, 26, 28, 19, 33, 29, 30,  2, 32, 19, 35, 18, 19, 12,
       12, 27,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0], dtype=uint8)

In [8]:
mapply.init(
    n_workers=-1,
    progressbar=True,
    chunk_size = 1024,
    max_chunks_per_worker = 32,
)

In [9]:
%%time
test_features = test_df['molecule'].to_pandas().mapply(get_char_idxs)
len(test_features)

100%|████████████████████████████████████████| 857/857 [00:54<00:00, 15.69it/s]



CPU times: user 11.1 s, sys: 58.9 s, total: 1min 9s
Wall time: 1min 9s


878022

In [10]:
train_features = train_df['molecule'].to_pandas().mapply(get_char_idxs)
len(train_features)

 98%|█████████████████████████████████████▏| 1284/1312 [20:14<00:27,  1.03it/s]


98415610

In [11]:
train_features = np.stack(train_features)
test_features = np.stack(test_features)
train_features.shape, test_features.shape

((98415610, 142), (878022, 142))

In [13]:
!mkdir -p /home/dangnh36/datasets/competitions/leash_belka/processed/features/char/

In [14]:
fp = np.memmap('/home/dangnh36/datasets/competitions/leash_belka/processed/features/char/train.mmap', dtype=train_features.dtype, mode='w+', shape=train_features.shape)
fp[:] = train_features[:]

In [15]:
fp = np.memmap('/home/dangnh36/datasets/competitions/leash_belka/processed/features/char/test.mmap', dtype=test_features.dtype, mode='w+', shape=test_features.shape)
fp[:] = test_features[:]

In [17]:
np.save('/home/dangnh36/datasets/competitions/leash_belka/processed/features/char/train.npy', train_features)

In [18]:
np.save('/home/dangnh36/datasets/competitions/leash_belka/processed/features/char/test.npy', test_features)

In [15]:
fp = np.memmap('/home/dangnh36/datasets/competitions/leash_belka/processed/features/char/train.npy', dtype=np.uint8, mode='r', shape=(98415610, 142))
fp[:500000, :].max()

36

In [16]:
fp[0]

memmap([ 8, 22,  8,  8, 28, 12, 27, 12, 12, 12, 17,  8, 33, 12, 18, 35,
        12, 17, 33,  8,  8,  4,  8,  8,  8, 33,  4, 12,  4, 12, 12, 12,
        35, 35,  4, 19, 35, 12, 17, 33, 29,  8,  3,  3,  5, 32, 17,  8,
         8, 22,  8, 19,  8,  8, 17, 26, 28, 19, 33, 29, 30,  2, 32, 19,
        35, 18, 19, 12, 12, 27,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       dtype=uint8)

In [17]:
np.array(fp[0])

array([ 8, 22,  8,  8, 28, 12, 27, 12, 12, 12, 17,  8, 33, 12, 18, 35, 12,
       17, 33,  8,  8,  4,  8,  8,  8, 33,  4, 12,  4, 12, 12, 12, 35, 35,
        4, 19, 35, 12, 17, 33, 29,  8,  3,  3,  5, 32, 17,  8,  8, 22,  8,
       19,  8,  8, 17, 26, 28, 19, 33, 29, 30,  2, 32, 19, 35, 18, 19, 12,
       12, 27,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0], dtype=uint8)